# Limpiando el dataset

## Imports necesarios para la limpieza y depuración del archivo

In [12]:
import csv
import datetime

## Lectura del archivo

In [13]:
data_raw = []

with open('../data/training_data_2_csv_UTF.csv', 'r') as csv_file_in:
    reader = csv.reader(csv_file_in, delimiter=',', quotechar='"')
    header = next(reader, None)
    for row in reader:
        data_raw.append(row)

original_size = len(data_raw)

## Filtrado de las columnas válidas

El archivo original tiene 24 posiciones pero solo las primeras 20 son representativas.

Lo primero que se debe hacer es quitar los últimos 4 elementos de las filas.

In [14]:
MAX_COLUMN = 20
header = header[0:MAX_COLUMN]
data_full = []

for row in data_raw:
    data_full.append(row[0:MAX_COLUMN])

header_with_position = []

for i in range(20):
    header_with_position.append(str(i) + ' - ' + header[i])

## Elección de las filas y guardado en nuevo archivo

Si alguna de las filas en la posición en la que esta definada la clase no termina ni con 1 o 0, no la tengo en cuenta.

En caso de querer agregar mas filtros para descartar filas, solo se debe agregar a la función `is_bad_row(a_row)`.

Para filtar columnas que no se quieren en el set datos final, se debe a agregar el número de columna en la lista `blacklist_column`.

La función `clean_row(a_row)` es la encargada de limpiar la fila y normalizar algunos de sus atributos.
Realza las siguientes acciones de limpieza:
* Quitar, a todas las celdas, las comillas extra que puedan tener.
* En la columna descripción (número 4) reemplaza el contenido de la misma por el valor Has content y por Has no content cuando es vacio.
* En la columan url (número 5) reemplaza el contenido por Url cuando hay un url presente, Empty cuando el contenido es vacio y None cuando el mismo es None o null.
* Normaliza la columna created_at y graba la cantidad de segundos desde la fecha indicada hasta el 1/1/2000. Hay tres tipos de fechas: 2/25/2015 20:11, 25/2/2015 20:11 o Sat Jun 07 19:34:43 +0000 2014.

In [15]:
'''
Las posiciones originales de los datos
'0 - id'
'1 - id_str'
'2 - screen_name'
'3 - location'
'4 - description'
'5 - url'
'6 - followers_count'
'7 - friends_count'
'8 - listed_count'
'9 - created_at'
'10 - favourites_count'
'11 - verified'
'12 - statuses_count'
'13 - lang'
'14 - status'
'15 - default_profile'
'16 - default_profile_image'
'17 - has_extended_profile'
'18 - name'
'19 - bot'
'''

LOCATION_POSITION = 3
DESCRIPTION_POSITION = 4
URL_POSITION = 5
CREATED_POSITION = 9
EXTENDED_PROFILE_POSITION = 17
CLASS_POSITION = 19
blacklist_column = [0, 1, 2, 14, 18]
posibles_date_formats = ["%a %b %d %H:%M:%S %z %Y", "%m/%d/%Y %H:%M", "%d/%m/%Y %H:%M"]
final_date_format = "%m/%d/%Y %H:%M"
date_2000=datetime.datetime.strptime("1/1/2000 0:00", "%m/%d/%Y %H:%M")
NULL_VALUE = 'NULL'

def is_bad_row(a_row):
    if row[CLASS_POSITION] == '1' or row[CLASS_POSITION] == '0':
        return False
    else:
        return True
    
def convert_string_date_to_total_seconds(a_string_date):
    for a_format in posibles_date_formats:
        try:
            actual_date = datetime.datetime.strptime(a_string_date, a_format).replace(tzinfo=None)
            difference = actual_date - date_2000
            return difference.seconds
        except:
            continue
    print ("La fecha no pudo ser leida: " + a_string_date)
    return None

def clean_row(a_row):
    a_clean_row = []
    for cell in a_row:
        a_clean_row.append(cell.replace('"', ''))
    
    if a_clean_row[LOCATION_POSITION].strip() == '':
        a_clean_row[LOCATION_POSITION] = NULL_VALUE
        
    if a_clean_row[EXTENDED_PROFILE_POSITION] in ('None', 'null', None, ''):
        a_clean_row[EXTENDED_PROFILE_POSITION] = NULL_VALUE
    
    if a_clean_row[DESCRIPTION_POSITION] in ('None', 'null', None):
        a_clean_row[DESCRIPTION_POSITION] = NULL_VALUE
    elif a_clean_row[DESCRIPTION_POSITION].strip() == '':
        a_clean_row[DESCRIPTION_POSITION] = 'No descrition'
    else:
        a_clean_row[DESCRIPTION_POSITION] = 'Has description'
    
    if a_clean_row[URL_POSITION] in ('None', 'null', None):
        a_clean_row[URL_POSITION] = NULL_VALUE
    elif a_clean_row[URL_POSITION].strip() == '':
        a_clean_row[URL_POSITION] = 'No url'
    else:
        a_clean_row[URL_POSITION] = 'Url'
        
    a_clean_row[CREATED_POSITION] = convert_string_date_to_total_seconds(a_clean_row[CREATED_POSITION])
    
    return a_clean_row

def filter_columns(a_row):
    a_filter_row = []
    for position in range(MAX_COLUMN):
        if not position in blacklist_column:
            a_filter_row.append(a_row[position])
    return a_filter_row

csv.register_dialect('my_dialect', delimiter = ',', quotechar='"')
actual_size = 0

with open('../data/newdata.csv', 'w') as csv_file_out:
    writer = csv.writer(csv_file_out, dialect='my_dialect')
    writer.writerow(filter_columns(header))
    for row in data_full:
        if not is_bad_row(row):
            refined_row = clean_row(row)
            filered_row = filter_columns(refined_row)
            writer.writerow(filered_row)
            actual_size = actual_size + 1

print ("Cantidad de registros originales: " + str(original_size))
print ("Cantidad de registros depurados: " + str(actual_size))

Cantidad de registros originales: 2797
Cantidad de registros depurados: 2770
